## Building Scorecards

This notebook runs through the high-level featues of building a scorecard. 

## Discretization

In [1]:
import sys
sys.path.insert(0, 'c:/projects/py-dev/scorecard')
from scorecard.scorecard import Scorecard
from scorecard.performance import BinaryPerformance
import seaborn as sns

df = sns.load_dataset('titanic')

Scorecard models often do not use continuous variables as-is. They are first discretized to create a set of bins. The discretization is performed by building a decision tree on each continuous variable and extracting the split points.

The `discretize` class method takes `kwargs` that are passed on to the sklearn DecisionTreeClassifier object. This allows the user to control how many bins are created and how many observations they contain. 

The `discretize` method takes a dataframe and a `Performance` object. Non-numeric fields are not discretized, but they are included in the scorecard for modeling. After calling `discretize` a `Scorecard` object is returned.

In [2]:
perf = BinaryPerformance(df.survived)
mod = Scorecard.discretize(df.drop(columns=['survived']), perf, max_leaf_nodes=6, min_samples_leaf=50)

## Adjusting Variables

The `Scorecard` object maintains a collection of variables that the user can refine using basic operations like expanding and collapsing bin levels, neutralizing levels, or applying constraints on how the fitted model coefficients must relate to one another.

The first step is to simply display the variable, though, and that is performed by calling the `display` method and passing in the name of the variable. Below we can see what the "pclass" variable looks like and the kind of information that is summarized.

Because the model was created using `BinaryPerformance`, summary information related to a binary target variable is shown. Using another performance type will display different information.

In [4]:
mod

In [5]:
mod.display_variable('pclass')

AttributeError: 'NoneType' object has no attribute 'x'

In [17]:
mod.display_variable('fare')

,N,# 0s,# 1s,1s Rate,% 0s,% 1s,WoE,IV,Constr
"(-inf, 10.48]",339.0,272.0,67.0,0.197640,0.495446,0.195906,-0.927822,0.277920,
"(10.48, 15.65]",136.0,83.0,53.0,0.389706,0.151184,0.154971,0.024739,0.000094,
"(15.65, 31.14]",194.0,101.0,93.0,0.479381,0.183971,0.271930,0.390767,0.034371,
"(31.14, 50.99]",63.0,43.0,20.0,0.317460,0.078324,0.058480,-0.292180,0.005798,
"(50.99, 74.38]",62.0,27.0,35.0,0.564516,0.049180,0.102339,0.732799,0.038955,
"(74.38, inf]",97.0,23.0,74.0,0.762887,0.041894,0.216374,1.641859,0.286471,
Missing,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,neu
Total,891.0,549.0,342.0,0.383838,1.000000,1.000000,0.000000,0.643609,


### Adjusting Bins

In [18]:
mod['fare'].collapse([4, 5])

In [19]:
mod.display_variable('fare')

,N,# 0s,# 1s,1s Rate,% 0s,% 1s,WoE,IV,Constr
"(-inf, 10.48]",339.0,272.0,67.0,0.197640,0.495446,0.195906,-0.927822,0.277920,
"(10.48, 15.65]",136.0,83.0,53.0,0.389706,0.151184,0.154971,0.024739,0.000094,
"(15.65, 31.14]",194.0,101.0,93.0,0.479381,0.183971,0.271930,0.390767,0.034371,
"(31.14, 50.99]",63.0,43.0,20.0,0.317460,0.078324,0.058480,-0.292180,0.005798,
"(50.99, inf]",159.0,50.0,109.0,0.685535,0.091075,0.318713,1.252613,0.285143,
Missing,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,neu
Total,891.0,549.0,342.0,0.383838,1.000000,1.000000,0.000000,0.603326,


In [20]:
mod['fare'].undo()

In [21]:
mod.display_variable('fare')

,N,# 0s,# 1s,1s Rate,% 0s,% 1s,WoE,IV,Constr
"(-inf, 10.48]",339.0,272.0,67.0,0.197640,0.495446,0.195906,-0.927822,0.277920,
"(10.48, 15.65]",136.0,83.0,53.0,0.389706,0.151184,0.154971,0.024739,0.000094,
"(15.65, 31.14]",194.0,101.0,93.0,0.479381,0.183971,0.271930,0.390767,0.034371,
"(31.14, 50.99]",63.0,43.0,20.0,0.317460,0.078324,0.058480,-0.292180,0.005798,
"(50.99, 74.38]",62.0,27.0,35.0,0.564516,0.049180,0.102339,0.732799,0.038955,
"(74.38, inf]",97.0,23.0,74.0,0.762887,0.041894,0.216374,1.641859,0.286471,
Missing,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,neu
Total,891.0,549.0,342.0,0.383838,1.000000,1.000000,0.000000,0.643609,


## Constraints

Constraints can be set for each variable as well. The Scorecard object can by index like a dictionary and passed the variable name. This grants access to the variable itself and all of it's methods. 

In [1]:
mod['fare'].increasing_constraints()
mod.display_variable('fare')

NameError: name 'mod' is not defined